<a href="https://colab.research.google.com/github/TrevorNgu/Number-Data-Model/blob/main/NumberDataTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!unzip "./drive/MyDrive/NumberData/10000" -d "/content"

Streaming output truncated to the last 5000 lines.
  inflating: /content/10000/5/5_101.jpg  
  inflating: /content/10000/5/5_102.jpg  
  inflating: /content/10000/5/5_103.jpg  
  inflating: /content/10000/5/5_104.jpg  
  inflating: /content/10000/5/5_105.jpg  
  inflating: /content/10000/5/5_106.jpg  
  inflating: /content/10000/5/5_107.jpg  
  inflating: /content/10000/5/5_108.jpg  
  inflating: /content/10000/5/5_109.jpg  
  inflating: /content/10000/5/5_11.jpg  
  inflating: /content/10000/5/5_110.jpg  
  inflating: /content/10000/5/5_111.jpg  
  inflating: /content/10000/5/5_112.jpg  
  inflating: /content/10000/5/5_113.jpg  
  inflating: /content/10000/5/5_114.jpg  
  inflating: /content/10000/5/5_115.jpg  
  inflating: /content/10000/5/5_116.jpg  
  inflating: /content/10000/5/5_117.jpg  
  inflating: /content/10000/5/5_118.jpg  
  inflating: /content/10000/5/5_119.jpg  
  inflating: /content/10000/5/5_12.jpg  
  inflating: /content/10000/5/5_120.jpg  
  inflating: /content/10000

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
import tensorflow as tf;
import tensorflow_datasets as tfds
import numpy as np
import PIL
import cv2
import os
import matplotlib.pyplot as plt
import random
from tqdm import tqdm
from keras.preprocessing import image
from keras.applications.resnet50 import decode_predictions
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.resnet50 import ResNet50
from keras.applications.vgg19 import VGG19
from keras.models import Sequential
from keras.applications import resnet50
from keras.layers import GlobalAveragePooling2D
from keras.layers import Dense, Flatten
from keras import Model
from keras.models import load_model
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

tf.debugging.experimental.enable_dump_debug_info(
    "/tmp/tfdbg2_logdir",
    tensor_debug_mode="FULL_HEALTH",
    circular_buffer_size=-1)

curr_model = None

def read_7_bytes(file):
  """Reads 7 bytes from a file and returns them as a NumPy array."""
  bytes = file.read(7)
  if len(bytes) < 7:
    return None
  return np.frombuffer(bytes, dtype=np.uint8)

def read_dir():
  path = "./train/"
  for dire in os.listdir(path):
    fpath = os.path.join(path, dire)
    file = open(fpath, "rb")

    header = file.readline().decode("utf-8").strip()
    width = file.readline().decode("utf-8").strip()
    height = file.readline().decode("utf-8").strip()
    bitsperfilter = file.readline().decode("utf-8").strip()
    diameter = file.readline().decode("utf-8").strip()
    vertspace = file.readline().decode("utf-8").strip()
    horspace = file.readline().decode("utf-8").strip()
    horfirst = file.readline().decode("utf-8").strip()
    vertfirst = file.readline().decode("utf-8").strip()

    print(header + " " + width + " " + height + " " + bitsperfilter + " " + diameter + " " + vertspace + " " + horspace + " " + horfirst + " " + vertfirst)

    array = np.array([])
    array.setflags(write=1)
    i = 0
    while True:
      i += 1
      bytes = read_7_bytes(file)
      bytes = np.copy(bytes)

      if bytes.all() == None:
        print(i)
        break

      #for i in range(7):
        #bytes[i] = (bytes[i] - 128)
        #bytes[i] = (bytes[i] * 23000)

      print(bytes)
      array = np.append(array, bytes)
    print(array)
    val = tf.convert_to_tensor(array, dtype="float32")
    print(val)

def image_files(input_directory):
    filepaths=[]
    labels=[]

    digit_folders=os.listdir(input_directory)
    #print(digit_folders)

    for digit in digit_folders:
        path=os.path.join(input_directory, digit)
        flist=os.listdir(path)
        for f in flist:
            fpath=os.path.join(path,f)
            filepaths.append(fpath)
            labels.append(digit)
    return filepaths,labels

def load_images(filepaths):
    images = []
    for i in tqdm(range(len(filepaths))):
        img = image.load_img(filepaths[i], target_size=(32,32,3))
        img = image.img_to_array(img)
        img.astype('float32')
        img = img/255
        print(img)
        images.append(img)

    images = np.array(images)
    return images

def createModel():
  global curr_model
  vgg19 = VGG19(weights = 'imagenet',
                include_top = False,
                input_shape=(32, 32, 3)
                )

  curr_model = Sequential()
  curr_model.add(vgg19)
  curr_model.add(Flatten())
  curr_model.add(Dense(10, activation='softmax'))

  curr_model.compile(loss='categorical_crossentropy',
                optimizer='sgd',
                metrics=['accuracy'])

  curr_model.summary()

def loadModel():
  global curr_model
  curr_model = tf.keras.models.load_model('./drive/MyDrive/NumberData/model/my_model.h5')
  curr_model.summary()

def saveModel():
  global curr_model
  if curr_model == None:
    print("No model currently")
  else:
    curr_model.save('./drive/MyDrive/NumberData/model')

def trainModel():
  directory = "./10000"
  filepaths, labels = image_files(directory)

  images = load_images(filepaths)

  y = to_categorical(labels, num_classes=10)
  x_train, x_test, y_train, y_test = train_test_split(images, y, random_state=42, test_size=0.2)

  print(x_train.shape)
  print(x_test.shape)
  curr_model.summary()

  history = curr_model.fit(x_train, y_train,
                           epochs=20,
                           batch_size=128,
                           validation_data=(x_test, y_test))
  curr_model.summary()

  curr_model.save('./drive/MyDrive/NumberData/model/my_model.h5')

def predict():
  global curr_model
  pathy = 'predict/'
  fname = input()
  pathy = pathy + fname
  img = image.load_img(pathy, target_size = (32, 32))
  img = image.img_to_array(img)
  img = np.expand_dims(img, axis = 0)

  prediction = curr_model.predict(img)
  print(prediction)

while True:
  print("1. Create Model\n2. Load Model\n3. Save Model\n4. Train with Model\n5. Predict with Model\n6. Exit")
  x = int(input())

  match x:
    case 1:
        createModel()
    case 2:
        loadModel()
    case 3:
        saveModel()
    case 4:
        trainModel()
    case 5:
        predict()
    case 6:
        break

Num GPUs Available:  0
1. Create Model
2. Load Model
3. Save Model
4. Train with Model
5. Predict with Model
6. Exit
4


  0%|          | 48/10000 [00:00<00:21, 473.25it/s]

[[[0.6509804  0.61960787 0.4745098 ]
  [0.64705884 0.6156863  0.4745098 ]
  [0.6392157  0.60784316 0.46666667]
  ...
  [0.6156863  0.5764706  0.43529412]
  [0.63529414 0.59607846 0.45490196]
  [0.6431373  0.5921569  0.45490196]]

 [[0.654902   0.62352943 0.47843137]
  [0.64705884 0.6156863  0.4745098 ]
  [0.6431373  0.6117647  0.47058824]
  ...
  [0.63529414 0.59607846 0.45490196]
  [0.64705884 0.60784316 0.46666667]
  [0.6313726  0.5803922  0.44313726]]

 [[0.65882355 0.627451   0.48235294]
  [0.6509804  0.61960787 0.47843137]
  [0.64705884 0.6156863  0.4745098 ]
  ...
  [0.63529414 0.59607846 0.45490196]
  [0.627451   0.5882353  0.44705883]
  [0.6392157  0.5882353  0.4509804 ]]

 ...

 [[0.64705884 0.6039216  0.47843137]
  [0.6392157  0.59607846 0.47058824]
  [0.64705884 0.6039216  0.47843137]
  ...
  [0.63529414 0.6        0.47058824]
  [0.6431373  0.60784316 0.47843137]
  [0.65882355 0.6313726  0.5176471 ]]

 [[0.6509804  0.6117647  0.4745098 ]
  [0.6509804  0.6117647  0.4745098 ]


  2%|▏         | 188/10000 [00:00<00:22, 441.49it/s]


[[[0.8039216  0.78431374 0.67058825]
  [0.8039216  0.78431374 0.67058825]
  [0.8039216  0.78431374 0.6666667 ]
  ...
  [0.76862746 0.74509805 0.6117647 ]
  [0.7647059  0.7411765  0.60784316]
  [0.7921569  0.76862746 0.63529414]]

 [[0.79607844 0.78039217 0.6666667 ]
  [0.79607844 0.78039217 0.6666667 ]
  [0.79607844 0.78431374 0.6627451 ]
  ...
  [0.7529412  0.7372549  0.6       ]
  [0.74509805 0.72156864 0.5803922 ]
  [0.7647059  0.7411765  0.6       ]]

 [[0.7921569  0.7764706  0.6627451 ]
  [0.7921569  0.7764706  0.6627451 ]
  [0.7921569  0.78039217 0.65882355]
  ...
  [0.7490196  0.73333335 0.59607846]
  [0.73333335 0.7176471  0.57254905]
  [0.7490196  0.73333335 0.5882353 ]]

 ...

 [[0.85490197 0.8039216  0.6666667 ]
  [0.8509804  0.8        0.6627451 ]
  [0.8352941  0.78431374 0.64705884]
  ...
  [0.8039216  0.78039217 0.654902  ]
  [0.7647059  0.7411765  0.6156863 ]
  [0.74509805 0.72156864 0.59607846]]

 [[0.85882354 0.81960785 0.6784314 ]
  [0.8509804  0.8117647  0.67058825]

  2%|▏         | 233/10000 [00:00<00:22, 443.33it/s]